In [6]:
import tensorflow as tf

x_data = [[1,2,1,1,],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes=3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    a = sess.run(hypothesis, feed_dict={X: [[1,11,7,9],[1,3,4,3],[1,1,0,1]]})
    print(a, sess.run(tf.argmax(a, 1)))


0 1.6082935
200 0.5061033
400 0.4200757
600 0.3463158
800 0.27302292
1000 0.22996303
1200 0.20862705
1400 0.19081046
1600 0.17570889
1800 0.16275126
2000 0.15151757
[[2.4455180e-03 9.9754637e-01 8.0783411e-06]
 [9.0512717e-01 8.6889818e-02 7.9830885e-03]
 [1.0573179e-08 3.0222823e-04 9.9969780e-01]] [1 0 2]


In [13]:
# Fancy Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                    X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2f}".format(step, loss, acc))
    
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (n,1) = flatten => (n, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot: Tensor("one_hot_6:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape_6:0", shape=(?, 7), dtype=float32)
Step:     0	Loss: 6.247	Acc: 0.06
Step:   100	Loss: 0.599	Acc: 0.88
Step:   200	Loss: 0.398	Acc: 0.91
Step:   300	Loss: 0.308	Acc: 0.93
Step:   400	Loss: 0.251	Acc: 0.95
Step:   500	Loss: 0.211	Acc: 0.95
Step:   600	Loss: 0.180	Acc: 0.95
Step:   700	Loss: 0.156	Acc: 0.95
Step:   800	Loss: 0.137	Acc: 0.96
Step:   900	Loss: 0.121	Acc: 0.98
Step:  1000	Loss: 0.109	Acc: 0.98
Step:  1100	Loss: 0.099	Acc: 0.98
Step:  1200	Loss: 0.090	Acc: 0.98
Step:  1300	Loss: 0.083	Acc: 1.00
Step:  1400	Loss: 0.077	Acc: 1.00
Step:  1500	Loss: 0.072	Acc: 1.00
Step:  1600	Loss: 0.067	Acc: 1.00
Step:  1700	Loss: 0.063	Acc: 1.00
Step:  1800	Loss: 0.059	Acc: 1.00
Step:  1900	Loss: 0.056	Acc: 1.00
[True] prediction: 0 True Y: 0
[True] prediction: 0 True Y: 0
[True] prediction: 3 True Y: 3
[True] prediction: 0 True Y: 0
[True] prediction: 0 True Y: 0
[True] predicti